In [3]:
import torch 
import torch.nn as nn 

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.up = nn.Linear(2,5) 
        self.act = nn.GELU()
        self.down = nn.Linear(5,2)
    
    def forward(self, x):
        x = self.up(x)
        x = self.act(x)
        x = self.down(x)
        return x

In [7]:


class HookWrapper():
    
    holder = [] 
    def __init__(self, model):
        self.model = model  
        self.hook = None 
        self.target_module = self.model.act
        
    def register_hooks(self):
        def hook_fn(module, input, output):
            return module.saved
        self.hook = self.target_module.register_forward_hook(hook_fn)
        
    def forward(self, x, other=None):
        self.register_hooks()
        self.target_module.saved = other
        x = self.model.forward(x)
        self.remove_hooks()
        return  x
    
    def remove_hooks(self):
        del self.hook
        
model = Model()
wrapper = HookWrapper(model)

x = torch.randn(1,2) 
other = torch.randn(1,5)*1000
wrapper.forward(x, other)

tensor([[-924.8879,  689.5286]], grad_fn=<AddmmBackward0>)

In [8]:
wrapper.forward(x)

tensor([[-0.2922, -0.1601]], grad_fn=<AddmmBackward0>)